In [2]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Literal
from dotenv import load_dotenv
from pydantic import BaseModel, Field

load_dotenv()

True

In [3]:
model = ChatOpenAI(model="gpt-4o-mini")

In [4]:
class SentimentSchema(BaseModel):

    sentiment: Literal["positive", "negative"] = Field(description="Sentiment of the review")

In [8]:
class DiagnosisSchema(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review')
    tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user')
    urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')

In [9]:
structured_model = model.with_structured_output(SentimentSchema)
structured_model2 = model.with_structured_output(DiagnosisSchema)

In [6]:
# state

class ReviewState(TypedDict):

    review: str
    sentiment: Literal["positive", "negative"]
    diagnosis: dict
    response: str

In [7]:
def find_sentiment(state: ReviewState):

    prompt = f"For the following review find out the sentiment \n {state["review"]}"
    sentiment = structured_model.invoke(prompt)

    return {"sentient": sentiment}

def check_sentiment(state: ReviewState) -> Literal["positive_response", "run_diagnosis"]:

    if state["sentiment"] == "positive":
        return "positive_response"
    else:
        return "run_diagnosis"

def positive_response(state: ReviewState):

    prompt = f"""Write a warm thank-you message in response to this review:
                 \n\n\"{state['review']}\"\n
                 Also, kindly ask the user to leave feedback on our website."""
    response = model.invoke(prompt)
    return {"response": response}

def run_diagnosis(state: ReviewState):

    prompt = f"""Diagnose this negative review:\n\n{state['review']}\n"
                "Return issue_type, tone, and urgency.
              """
    response = structured_model2.invoke(prompt).content

    return {"diagnosis": response.model_dump()} # convert the pydantic object to python dictionary

def negative_response(state: ReviewState):

    diagnosis = state["diagnosis"]
    prompt = f"""You are a support assistant.
                 The user had a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}', and marked urgency as '{diagnosis['urgency']}'.
                 Write an empathetic, helpful resolution message.
              """
    response = model.invoke(prompt).content
    return {"response": response}

In [ ]:
# graph
graph = StateGraph(ReviewState)

# nodes
graph.add_node("find_sentiment", find_sentiment)

# edges
graph.add_edge(START, "find_sentiment")


workflow = graph.compile()